In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

import pandas as pd
! pip install transformers
! pip install sentencepiece
from transformers import pipeline
summarizer = pipeline("summarization", model="google/pegasus-xsum", device = 0)
sentiment_tagger = pipeline("sentiment-analysis", model="ProsusAI/finbert", device = 0)
'''
sentiment_translator = {
  "Bullish": "positive",
  "Bearish": "negative",
  "Neutral": "neutral"
}
'''


#human-centered-summarization/financial-summarization-pegasus
# facebook/bart-large-cnn


# add Firebase credentials here


# the coins list needs to be imported 
coins_list = pd.read_csv('/content/coins.csv')

In [ ]:
# news classification function:

def categorize(title):
  contained_coins = []
  for index in coins_list.index:
    if coins_list['name'][index] in title.split():
      contained_coins.append(coins_list['name'][index])
    if coins_list['symbol'][index] in title.split():
      contained_coins.append(coins_list['symbol'][index])
    if coins_list['slug'][index] in title.split():
      contained_coins.append(coins_list['slug'][index])
  return contained_coins


# function for checking if the article is Price Prediction or not:
def is_price_prediction(title):
  if 'price prediction' in title.lower():
    return "yes"
  else:
    return "no"


# function for checking if the article is newsletter or not:
def is_news_letter(title):
  if 'newsletter' in title.lower():
    return "yes"
  else:
    return "no"

In [ ]:
# instead of para by para, give it all to the model:

# getting articles from Cointelegraph, categorizing, and checking for price prediction and newsletter:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

url = 'https://cointelegraph.com/category/latest-news'
driver.get(url)

articles = driver.find_elements(By.CLASS_NAME, "post-card-inline")
#####links_list = []
for article in articles:
  title_tag = article.find_element(By.XPATH, ".//*[@class = 'post-card-inline__title-link']")
  title = title_tag.text
  if len(categorize(title)) != 0:
    if (is_price_prediction(title) == 'no') & (is_news_letter(title) == 'no'):
      link = title_tag.get_attribute('href')
      print(title)
      # push to firebase here: Push the tile, the link, and the source (Cointelegraph).
      ######links_list.append(link)
      driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      driver.get(link)
      tag = WebDriverWait(driver, 25).until(
      EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='post-content']//p")))
      #print(len(tag))
      i = 1
      summary = ' '
      full_text = ' '
      while i < len(tag):
        try:
          text = WebDriverWait(driver, 50).until(
          EC.presence_of_element_located((By.XPATH, f".//div[@class='post-content']//p[{i}]"))).text
          if (len(text.split()) >= 5) & ('Related: ' not in text):
            print('paragraph: ',text)
            print('lengh of the paragraph is: ', len(text.split()))
            full_text = full_text+ text + ". "

            # max_length= round(0.8*len(text.split())), min_length= round(0.2*len(text.split())), 
          i+=1
        except TimeoutException:
          #print("Done")
          break
  try:
    print('full_text is: ', full_text) 
    print('full_text lenthg is: ', len(full_text.split())) 
    summary = summarizer(full_text,max_length= round(0.7*len(full_text.split())), min_length= round(0.2*len(full_text.split())), do_sample=False)[0]["summary_text"] 
    print("lenght of summary is: ", len(summary.split()))
    print('summary is: ', summary)
    sentiment = sentiment_tagger(summary, max_length=512, truncation = True)[0]['label']
    print('sentiment is: ', sentiment) 
    print("_"*80) 

  except NameError:
    continue


In [ ]:
######### test



# instead of para by para, give it all to the model:

# getting articles from Cointelegraph, categorizing, and checking for price prediction and newsletter:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

url = 'https://cointelegraph.com/category/latest-news'
driver.get(url)

articles = driver.find_elements(By.CLASS_NAME, "post-card-inline")
#####links_list = []
for article in articles:
  title_tag = article.find_element(By.XPATH, ".//*[@class = 'post-card-inline__title-link']")
  title = title_tag.text
  print(title)
  print(title_tag.get_attribute('href'))
  print(len(categorize(title)))
  if len(categorize(title)) != 0:
    if (is_price_prediction(title) == 'no') & (is_news_letter(title) == 'no'):
      link = title_tag.get_attribute('href')
      # push to firebase here: Push the tile, the link, and the source (Cointelegraph).
      ######links_list.append(link)
      driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      driver.get(link)
      tag = WebDriverWait(driver, 25).until(
      EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='post-content']//p")))
      #print(len(tag))
      i = 1
      summary = ' '
      full_text = ' '
      while i < len(tag):
        try:
          text = WebDriverWait(driver, 50).until(
          EC.presence_of_element_located((By.XPATH, f".//div[@class='post-content']//p[{i}]"))).text
          if (len(text.split()) >= 5) & ('Related: ' not in text):
            print('paragraph: ',text)
            print('lengh of the paragraph is: ', len(text.split()))
            full_text = full_text+ text + ". "

            # max_length= round(0.8*len(text.split())), min_length= round(0.2*len(text.split())), 
          i+=1
        except TimeoutException:
          #print("Done")
          break
  try:
    print('full_text is: ', full_text) 
    print('full_text lenthg is: ', len(full_text.split())) 
    summary = summarizer(full_text,max_length= round(0.7*len(full_text.split())), min_length= round(0.2*len(full_text.split())), do_sample=False)[0]["summary_text"] 
    print("lenght of summary is: ", len(summary.split()))
    print('summary is: ', summary)
    sentiment = sentiment_tagger(summary, max_length=512, truncation = True)[0]['label']
    print('sentiment is: ', sentiment) 
    print("_"*80) 

  except NameError:
    continue
